In [ ]:
import json
with open('/content/fact_answerable_1sent.txt','r') as fp:
    text_fact11 = json.load(fp)

In [ ]:
import json
with open('/content/fact_ans_1_sent_test1.txt','r') as fp:
    text_fact111 = json.load(fp)

In [ ]:
text_fact11['data'][0]['paragraphs'][0]

{'qas': [{'id': '56bc751eac7ad10019000013_1',
   'question': 'Name synonym of Acrokeratosis paraneoplastica.',
   'answers': [{'text': 'bazex syndrome', 'answer_start': 31}],
   'is_available': True}],
 'context': 'acrokeratosis paraneoplastica (bazex syndrome).a 55-year-old white man born in canada presented with all the clinical features of acrokeratosis paraneoplastica of bazex.'}

In [ ]:
text_fact111['data'][0]['paragraphs'][0]

{'qas': [{'id': '601bde6e1cb411341a000006_1',
   'question': 'Which disease is caused by de novo VPS4A mutations?',
   'answers': [{'text': 'multisystem disease with abnormal neurodevelopment',
     'answer_start': 30}],
   'is_available': True}],
 'context': 'de novo vps4a mutations cause multisystem disease with abnormal neurodevelopment.the endosomal sorting complexes required for transport (escrts) are essential for multiple membrane modeling and membrane-independent cellular processes.'}

In [ ]:
def read_squad(path):
    with open(path, 'rb') as f:
        squad_dict = json.load(f)

    # initialize lists for contexts, questions, and answers
    contexts = []
    questions = []
    answers = []
    # iterate through all data in squad data
    for group in squad_dict['data']:
        for passage in group['paragraphs']:
            context = passage['context']
            for qa in passage['qas']:
                question = qa['question']
                if 'plausible_answers' in qa.keys():
                    access = 'plausible_answers'
                else:
                    access = 'answers'
                for answer in qa['answers']:
                    # append data to lists
                    contexts.append(context)
                    questions.append(question)
                    answers.append(answer)
    # return formatted data lists
    return contexts, questions, answers

In [ ]:
val_contexts, val_questions, val_answers = read_squad('/content/fact_ans_1_sent_test.txt')
train_contexts, train_questions, train_answers = read_squad('/content/fact_answerable_1sent.txt')

In [ ]:
val_contexts, val_questions, val_answers = read_squad('/content/fact_ans_1_sent_test1.txt')

In [ ]:
def add_end_idx(answers, contexts):
    # loop through each answer-context pair
    for answer, context in zip(answers, contexts):
        # gold_text refers to the answer we are expecting to find in context
        gold_text = answer['text']
        # we already know the start index
        start_idx = answer['answer_start']
        # and ideally this would be the end index...
        end_idx = start_idx + len(gold_text)

        # ...however, sometimes squad answers are off by a character or two
        if context[start_idx:end_idx] == gold_text:
            # if the answer is not off :)
            answer['answer_end'] = end_idx
        else:
            for n in [1, 2]:
                if context[start_idx-n:end_idx-n] == gold_text:
                    # this means the answer is off by 'n' tokens
                    answer['answer_start'] = start_idx - n
                    answer['answer_end'] = end_idx - n

In [ ]:
add_end_idx(train_answers, train_contexts)
add_end_idx(val_answers, val_contexts)

In [ ]:
add_end_idx(val_answers, val_contexts)

In [ ]:
!pip install transformers

In [ ]:
from transformers import AutoTokenizer, AutoModelForQuestionAnswering

tokenizer = AutoTokenizer.from_pretrained("franklu/pubmed_bert_squadv2",model_max_length=512)
#tokenizer = AutoTokenizer.from_pretrained("microsoft/BiomedNLP-PubMedBERT-base-uncased-abstract-fulltext",model_max_length=512)
train_encodings = tokenizer(train_contexts, train_questions, truncation=True, padding=True)
val_encodings = tokenizer(val_contexts, val_questions, truncation=True, padding=True)

In [ ]:
val_encodings = tokenizer(val_contexts, val_questions, truncation=True, padding=True)

In [ ]:
def add_token_positions(encodings, answers):
    # initialize lists to contain the token indices of answer start/end
    start_positions = []
    end_positions = []
    for i in range(len(answers)):
        # append start/end token position using char_to_token method
        start_positions.append(encodings.char_to_token(i, answers[i]['answer_start']))
        end_positions.append(encodings.char_to_token(i, answers[i]['answer_end']))

        # if start position is None, the answer passage has been truncated
        if start_positions[-1] is None:
            start_positions[-1] = tokenizer.model_max_length
        # end position cannot be found, char_to_token found space, so shift one token forward
        go_back = 1
        while end_positions[-1] is None:
            end_positions[-1] = encodings.char_to_token(i, answers[i]['answer_end']-go_back)
            go_back +=1
    # update our encodings object with the new token-based start/end positions
    encodings.update({'start_positions': start_positions, 'end_positions': end_positions})

# apply function to our data
add_token_positions(train_encodings, train_answers)
add_token_positions(val_encodings, val_answers)

In [ ]:
add_token_positions(val_encodings, val_answers)

In [ ]:
import torch

class SquadDataset(torch.utils.data.Dataset):
    def __init__(self, encodings):
        self.encodings = encodings

    def __getitem__(self, idx):
        return {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}

    def __len__(self):
        return len(self.encodings.input_ids)

train_dataset = SquadDataset(train_encodings)
val_dataset = SquadDataset(val_encodings)

In [ ]:
val_dataset = SquadDataset(val_encodings)

In [ ]:
from transformers import AutoTokenizer, AutoModelForQuestionAnswering

model = AutoModelForQuestionAnswering.from_pretrained("franklu/pubmed_bert_squadv2")
#model = AutoModelForMaskedLM.from_pretrained("microsoft/BiomedNLP-PubMedBERT-base-uncased-abstract-fulltext")

In [ ]:
!pip install madgrad

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from torch.utils.data import DataLoader
from transformers import AdamW
from tqdm import tqdm
from madgrad import MADGRAD

# setup GPU/CPU
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
# move model over to detected device
model.to(device)
# activate training mode of model
model.train()
# initialize adam optimizer with weight decay (reduces chance of overfitting)
#optim = AdamW(model.parameters(), lr=5e-5)
optim = AdamW(model.parameters(), lr=5e-5)
#optim = MADGRAD(model.parameters(), lr=5e-7)
# initialize data loader for training data
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)

for epoch in range(1):
    # set model to train mode
    model.train()
    # setup loop (we use tqdm for the progress bar)
    loop = tqdm(train_loader, leave=True)
    for batch in loop:
        # initialize calculated gradients (from prev step)
        optim.zero_grad()
        # pull all the tensor batches required for training
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        start_positions = batch['start_positions'].to(device)
        end_positions = batch['end_positions'].to(device)
        # train model on batch and return outputs (incl. loss)
        outputs = model(input_ids, attention_mask=attention_mask,
                        start_positions=start_positions,
                        end_positions=end_positions)
        # extract loss
        loss = outputs[0]
        # calculate loss for every parameter that needs grad update
        loss.backward()
        # update parameters
        optim.step()
        # print relevant info to progress bar
        loop.set_description(f'Epoch {epoch}')
        loop.set_postfix(loss=loss.item())

Epoch 0: 100%|██████████| 839/839 [03:40<00:00,  3.81it/s, loss=0.491]


In [ ]:
model_path = 'models3/distilbert-custom'
model.save_pretrained(model_path)
tokenizer.save_pretrained(model_path)

('models3/distilbert-custom/tokenizer_config.json',
 'models3/distilbert-custom/special_tokens_map.json',
 'models3/distilbert-custom/vocab.txt',
 'models3/distilbert-custom/added_tokens.json',
 'models3/distilbert-custom/tokenizer.json')

In [ ]:
model.eval()

#val_sampler = SequentialSampler(val_dataset)
val_loader = DataLoader(val_dataset, batch_size=16)

acc = []

# initialize loop for progress bar
loop = tqdm(val_loader)
# loop through batches
for batch in loop:
    # we don't need to calculate gradients as we're not training
    with torch.no_grad():
        # pull batched items from loader
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        start_true = batch['start_positions'].to(device)
        end_true = batch['end_positions'].to(device)
        # make predictions
        outputs = model(input_ids, attention_mask=attention_mask)
        # pull preds out
        start_pred = torch.argmax(outputs['start_logits'], dim=1)
        end_pred = torch.argmax(outputs['end_logits'], dim=1)
        # calculate accuracy for both and append to accuracy list
        acc.append(((start_pred == start_true).sum()/len(start_pred)).item())
        acc.append(((end_pred == end_true).sum()/len(end_pred)).item())
# calculate average accuracy in total
acc1 = sum(acc)/len(acc)


In [ ]:
acc1 

In [ ]:
acc


In [ ]:
print("T/F\tstart\tend\n")
for i in range(len(start_true)):
    print(f"true\t{start_true[i]}\t{end_true[i]}\n"
          f"pred\t{start_pred[i]}\t{end_pred[i]}\n")

T/F	start	end

true	3	4
pred	4	4

true	4	5
pred	4	5

true	3	5
pred	3	5

true	6	10
pred	6	7

